## Imports

In [1]:
import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import matplotlib.pyplot as plt
import os
import warnings

warnings.simplefilter("ignore", category=UserWarning)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


## Environment

In [28]:
import gymnasium_robotics

env = gym.make("PointMaze_UMaze-v3", max_episode_steps=100, render_mode = "human")

In [29]:
env.reset()

({'observation': array([ 0.84609324, -0.82185403,  0.        ,  0.        ]),
  'achieved_goal': array([ 0.84609324, -0.82185403]),
  'desired_goal': array([-0.82334061, -1.00952485])},
 {'success': False})

In [30]:
env.action_space

Box(-1.0, 1.0, (2,), float32)

In [31]:
env.observation_space

Dict('achieved_goal': Box(-inf, inf, (2,), float64), 'desired_goal': Box(-inf, inf, (2,), float64), 'observation': Box(-inf, inf, (4,), float64))

In [32]:
env.render()

In [ ]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset(seed=42)
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, truncated, info = env.step(action)
        score += reward
    print(f"Episode: {i} Score: {score}")

env.close()

## Train Model

In [34]:
env = DummyVecEnv([lambda: env])

In [37]:
log_path = os.path.join("Training", "logs")
model = PPO("MultiInputPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [38]:
model.learn(total_timesteps=100000)

Logging to Training/logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 69   |
|    iterations      | 1    |
|    time_elapsed    | 29   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 61          |
|    iterations           | 2           |
|    time_elapsed         | 66          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014204145 |
|    clip_fraction        | 0.133       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.84       |
|    explained_variance   | 0.566       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0161     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0165     |
|    std                  | 1           |
|    value_loss           | 0.00916     |
---

Use "tensorboard --logdir logs/" in the Training terminal to get to tensorboard

## Test Model

In [41]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(40.9, 30.771577795101763)

In [42]:
episodes = 5
for i in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward        
    
    print(f"Episode: {i} Score: {score}")

env.close()

Episode: 1 Score: [2.]
Episode: 2 Score: [0.]
Episode: 3 Score: [5.]
Episode: 4 Score: [20.]
Episode: 5 Score: [74.]
